In [26]:
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn.preprocessing import LabelEncoder

In [57]:
transactions_path = '/home/mahboob/Downloads/fct_data/transactions.csv'
fraudsters_path = '/home/mahboob/Downloads/fct_data/fraudsters.csv'
users_path = '/home/mahboob/Downloads/fct_data/users.csv'
new_fraud_file_path = '/home/mahboob/Downloads/fct_data/updated_fraudsters.csv'

In [3]:
transactions_df = pd.read_csv(transactions_path)
fraudsters_df = pd.read_csv(fraudsters_path)
users_df = pd.read_csv(users_path)

In [4]:
le = LabelEncoder()
transactions_df['CURRENCY'] = le.fit_transform(transactions_df['CURRENCY'].astype('str'))
transactions_df['STATE'] = le.fit_transform(transactions_df['STATE'].astype('str'))
transactions_df['MERCHANT_CATEGORY'] = le.fit_transform(transactions_df['MERCHANT_CATEGORY'].astype('str'))
transactions_df['ENTRY_METHOD'] = le.fit_transform(transactions_df['ENTRY_METHOD'].astype('str'))
transactions_df['TYPE'] = le.fit_transform(transactions_df['TYPE'].astype('str'))
transactions_df['SOURCE'] = le.fit_transform(transactions_df['SOURCE'].astype('str'))
transactions_df['MERCHANT_COUNTRY'] = le.fit_transform(transactions_df['MERCHANT_COUNTRY'].astype('str'))

In [5]:
from datetime import datetime
#date = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S:%f")
month = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S.%f" ).month
week_day = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S.%f" ).weekday()
times_of_day = [0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 5, 6, 6, 6, 7, 7, 7 ]
time_of_day = lambda x: times_of_day[datetime.strptime(x, "%Y-%m-%d %H:%M:%S.%f").hour]

transactions_df['month'] = transactions_df['CREATED_DATE'].map(month)
transactions_df['day_of_week'] = transactions_df['CREATED_DATE'].map(week_day)
transactions_df['time_of_day'] = transactions_df['CREATED_DATE'].map(month)

In [6]:
fraud_transactions = pd.merge(fraudsters_df, transactions_df, left_on = 'user_id', right_on = 'USER_ID', how='left')

In [22]:
fraud_transactions.head()

,CURRENCY,AMOUNT,STATE,MERCHANT_CATEGORY,MERCHANT_COUNTRY,ENTRY_METHOD,TYPE,SOURCE,month,day_of_week,time_of_day
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9.0,59700.0,1.0,73.0,362.0,5.0,4.0,8.0,6.0,4.0,6.0
2,9.0,10000.0,1.0,73.0,362.0,5.0,1.0,8.0,6.0,4.0,6.0
3,9.0,10000.0,1.0,73.0,362.0,5.0,1.0,2.0,7.0,6.0,7.0
4,9.0,10000.0,1.0,73.0,362.0,5.0,1.0,8.0,7.0,0.0,7.0


In [21]:
fraud_transactions.drop(fraud_transactions.columns[0], axis=1, inplace=True)

In [25]:
fraud_transactions.shape

(14544, 11)

In [28]:
fraud_transactions = fraud_transactions.dropna()

In [29]:
clf = svm.OneClassSVM(nu=0.1, kernel="rbf", gamma=0.1)
clf.fit(fraud_transactions)

OneClassSVM(cache_size=200, coef0=0.0, degree=3, gamma=0.1, kernel='rbf',
      max_iter=-1, nu=0.1, random_state=None, shrinking=True, tol=0.001,
      verbose=False)

In [40]:
transactions_df.head()

,CURRENCY,AMOUNT,STATE,CREATED_DATE,MERCHANT_CATEGORY,MERCHANT_COUNTRY,ENTRY_METHOD,USER_ID,TYPE,SOURCE,ID,month,day_of_week,time_of_day
0,9,3738,1,2015-10-11 09:05:43.016000,13,122,5,7285c1ec-31d0-4022-b311-0ad9227ef7f4,2,3,5a9ee109-e9b3-4598-8dd7-587591e6a470,10,6,10
1,9,588,1,2015-10-11 20:08:39.150000,73,149,5,20100a1d-12bc-41ed-a5e1-bc46216e9696,2,3,28d68bf4-460b-4c8e-9b95-bcda9ab596b5,10,6,10
2,9,1264,1,2015-10-11 11:37:40.908000,73,341,5,0fe472c9-cf3e-4e43-90f3-a0cfb6a4f1f0,2,3,1f1e8817-d40b-4c09-b718-cfc4a6f211df,10,6,10
3,9,66,6,2015-10-11 20:08:35.310000,73,149,5,20100a1d-12bc-41ed-a5e1-bc46216e9696,2,3,a7aaf78c-d201-456f-9e6d-612a795e8c32,10,6,10
4,9,968,1,2015-10-11 02:46:47.640000,108,282,5,821014c5-af06-40ff-91f4-77fe7667809f,2,3,27dd99a2-5539-4ba9-876a-1a94abc2701f,10,6,10


In [32]:
fraud_transactions_pred = clf.predict(fraud_transactions)

In [35]:
fraud_transactions_pred

array([-1,  1,  1, ...,  1, -1, -1])

In [36]:
np.array(np.unique(fraud_transactions_pred, return_counts=True)).T

array([[   -1,  4226],
       [    1, 10317]])

In [41]:
transactions_df.drop(transactions_df.columns[3], axis=1, inplace=True)

In [43]:
test_df = transactions_df[['CURRENCY', 'AMOUNT', 'STATE', 'MERCHANT_CATEGORY', 'MERCHANT_COUNTRY', 'ENTRY_METHOD', \
         'TYPE', 'SOURCE', 'month', 'day_of_week', 'time_of_day']]
test_df.columns
test_df.head()

,CURRENCY,AMOUNT,STATE,MERCHANT_CATEGORY,MERCHANT_COUNTRY,ENTRY_METHOD,TYPE,SOURCE,month,day_of_week,time_of_day
0,9,3738,1,13,122,5,2,3,10,6,10
1,9,588,1,73,149,5,2,3,10,6,10
2,9,1264,1,73,341,5,2,3,10,6,10
3,9,66,6,73,149,5,2,3,10,6,10
4,9,968,1,108,282,5,2,3,10,6,10


In [44]:
all_fraud_transactions_pred = clf.predict(test_df)

In [45]:
all_users = transactions_df['USER_ID']

In [49]:
new_fraud_users = []
for i, user in enumerate(all_users):
    if all_fraud_transactions_pred[i] == 1:
        new_fraud_users.append(user)

In [52]:
new_fraud_users = list(set(new_fraud_users))

In [53]:
len(new_fraud_users)

6139

In [54]:
new_fraud_users_df = pd.DataFrame(new_fraud_users)

In [58]:
new_fraud_users_df.to_csv(new_fraud_file_path)